<img src="https://industrial.uniandes.edu.co/sites/default/files/imagenes/uniandeslogo.png" alt="Universidad de los Andes" style="float: right; width: 300px; height: auto;">

# Taller 1 - Ejercicio 3

Econometría Avanzada, 2026-1  

Febrero de 2026

````
Aclaración: En este cuaderno se utilizo IA generativa, Claude, para ordenar, unificar, comentar el código y crear las tablas que se exportan a .tex
````

### Importar librerías

Carga las librerías necesarias para el análisis.

**Librerías utilizadas:**
- `pandas` y `numpy`: manipulación y análisis de datos
- `statsmodels`: estimación OLS con errores estándar clusterizados
- `yaml` y `pathlib`: gestión de rutas de archivos
- `warnings`: control de advertencias

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
import yaml
from pathlib import Path

### Configuración de rutas

Carga los directorios principales desde el archivo de configuración `paths.yml`.

In [2]:
with open('paths.yml', 'r') as f:
    paths = yaml.safe_load(f)

raw    = Path(paths['data']['raw'])
tables = Path(paths['outputs']['tables'])
tables.mkdir(parents=True, exist_ok=True)

### Configuración de opciones

Suprime advertencias durante la ejecución.

In [3]:
warnings.filterwarnings("ignore")

### Lectura de datos

Lee los archivos de encuesta de línea base (2018) y seguimiento (2021). Ambos corresponden al 60 % de los datos originales de Dupas et al. (2025).

In [4]:
df_baseline = pd.read_stata(raw / 'baseline_sample.dta')
df_endline  = pd.read_stata(raw / 'endline_sample.dta')

## Punto 4. Replicación de balance muestral (Tabla A.1)

### 4a. Construcción de variables analíticas

Construir las siete variables de la Tabla A.1 a partir del cuestionario de línea base.

In [5]:
df_b = df_baseline.copy()

# Renombrar variables para el análisis
df_b['age']              = df_b['an_bw_s3q5_age']
df_b['polygamous']       = df_b['an_bw_polygamous'].astype(float)
df_b['desired_children'] = df_b['an_bw_s8q1_childdesired']
df_b['modern_contra']    = df_b['an_bw_modern_cmeth_curr_s11q5']
df_b['unmet_need']       = df_b['an_unmet']
df_b['cannot_afford']    = df_b['an_able_contra']  # 1=no puede pagar (variable ya codificada como indicador)  # an_able_contra=1: puede pagar
df_b['has_radio']        = df_b['bw_s2q19a_radioyn']

balance_vars = [
    'age', 'polygamous', 'desired_children', 'modern_contra',
    'unmet_need', 'cannot_afford', 'has_radio'
]
balance_labels = [
    'Edad de la mujer',
    'Esposo poligamo',
    'Numero total de hijos deseados',
    'Uso actual de anticoncepcion moderna',
    'Necesidad insatisfecha de anticonceptivos',
    'No podria pagar por anticonceptivos',
    'El hogar tiene radio'
]

# Estadísticas descriptivas
df_b[balance_vars].describe().round(3)

,age,polygamous,desired_children,modern_contra,unmet_need,cannot_afford,has_radio
count,8687.000,8689.000,7855.000,8680.000,8682.000,7873.000,8686.000
mean,28.281,0.454,6.016,0.314,0.387,0.403,0.482
std,5.452,0.498,1.885,0.464,0.487,0.491,0.500
min,18.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,24.000,0.000,5.000,0.000,0.000,0.000,0.000
50%,28.000,0.000,6.000,0.000,0.000,0.000,0.000
75%,33.000,1.000,7.000,1.000,1.000,1.000,1.000
max,38.000,1.000,20.000,1.000,1.000,1.000,1.000


### 4d. Estadísticas descriptivas de la muestra

Calcula media, desviación estándar y N para la muestra completa, más la media del grupo control, para cada variable de balance. Se exporta como tabla LaTeX (tabular body sin float) para incluir en la respuesta del inciso 4d.

In [6]:

# Estadísticas descriptivas por variable de balance
desc_rows = []
for var, label in zip(balance_vars, balance_labels):
    s    = df_b[var].dropna()
    ctrl = df_b.loc[df_b['arm_free'] == 0, var].dropna()
    trat = df_b.loc[df_b['arm_free'] == 1, var].dropna()
    desc_rows.append({
        'Variable':   label,
        'N':          len(s),
        'Media':      round(s.mean(), 3),
        'DE':         round(s.std(),  3),
        'Media ctrl': round(ctrl.mean(), 3),
        'Media trat': round(trat.mean(), 3),
    })

df_desc = pd.DataFrame(desc_rows).set_index('Variable')

# Exportar tabular body (sin \begin{table} para poder incluir dentro de tcolorbox)
lines = [
    r'\begin{center}',
    r'\small',
    r'\begin{tabular}{lrrrrrr}',
    r'\hline\hline',
    r'Variable & N & Media & DE & Media ctrl & Media trat \\',
    r'\hline',
]
for r in desc_rows:
    label_esc = r['Variable'].replace('&', r'\&')
    lines.append(
        f"{label_esc} & {r['N']:,} & {r['Media']:.3f} & {r['DE']:.3f}"
        f" & {r['Media ctrl']:.3f} & {r['Media trat']:.3f} \\\\"
    )
lines += [
    r'\hline\hline',
    (r'\multicolumn{6}{l}{\footnotesize'
     r' Muestra de línea base (submuestra del 60\%). DE\,=\,desviación estándar.'
     r' Media ctrl\,=\,media del grupo de subsidio 10\%. Media trat\,=\,media del grupo de subsidio 100\%.} \\'),
    r'\end{tabular}',
    r'\end{center}',
]

with open(tables / 'tabla_descriptivas.tex', 'w', encoding='utf-8') as f:
    f.write('\n'.join(lines) + '\n')


### 4b. Regresiones de balance

Estima para cada variable $X_i$ la regresión:

$$X_i = \alpha + \beta \cdot \text{Tratamiento}_i + \gamma_{\text{provincia}} + \varepsilon_i$$

con errores estándar clusterizados a nivel de centro de salud (`new_csps`, 100 centros).

In [7]:
# Efectos fijos de provincia
province_dummies = pd.get_dummies(
    df_b['new_province'], prefix='prov', drop_first=True
).astype(float)

Función auxiliar para OLS con efectos fijos de provincia y errores estándar clusterizados. Se define una sola función que se reutiliza para las regresiones de balance (Q4) y las regresiones ITT (Q5).

In [8]:
def run_reg_clustered(y_col, df, treatment_col, prov_dummies, cluster_col):
    y    = df[y_col]
    X    = pd.concat([df[[treatment_col]], prov_dummies], axis=1).astype(float)
    X    = sm.add_constant(X)
    mask = y.notna() & X.notna().all(axis=1) & df[cluster_col].notna()
    model = sm.OLS(y[mask], X[mask]).fit(
        cov_type='cluster',
        cov_kwds={'groups': df[cluster_col][mask].values}
    )
    return model, mask

Ejecuta las regresiones de balance para cada variable. Extrae el coeficiente del tratamiento, el error estándar, el p-valor, la media del grupo control y el tamaño de muestra.

In [9]:
results_balance = []
for var, label in zip(balance_vars, balance_labels):
    model, mask = run_reg_clustered(
        var, df_b, 'arm_free', province_dummies, 'new_csps'
    )
    results_balance.append({
        'Variable':      label,
        'coef':          model.params['arm_free'],
        'se':            model.bse['arm_free'],
        'pval':          model.pvalues['arm_free'],
        'mean_ctrl':     df_b.loc[df_b['arm_free'] == 0, var].mean(),
        'N':             int(mask.sum())
    })

### F-estadístico conjunto

Calcula el estadístico F para la hipótesis de que todos los coeficientes de las variables de balance son iguales a cero de manera conjunta. Regresa la variable de tratamiento sobre las siete características de línea base más los efectos fijos de provincia.

In [10]:
mask_all = (
    df_b[balance_vars + ['arm_free', 'new_csps', 'new_province']]
    .notna().all(axis=1)
)

X_joint = pd.concat(
    [df_b.loc[mask_all, balance_vars].astype(float),
     province_dummies[mask_all]],
    axis=1
)
X_joint  = sm.add_constant(X_joint)
y_treat  = df_b.loc[mask_all, 'arm_free'].astype(float)

model_joint = sm.OLS(y_treat, X_joint).fit(
    cov_type='cluster',
    cov_kwds={'groups': df_b.loc[mask_all, 'new_csps'].values}
)

# Restricción: coeficientes de las variables de balance = 0
balance_idx = [model_joint.params.index.get_loc(v) for v in balance_vars]
R = np.zeros((len(balance_idx), len(model_joint.params)))
for i, idx in enumerate(balance_idx):
    R[i, idx] = 1.0

f_result = model_joint.f_test(R)
f_stat   = float(f_result.fvalue)
f_pval   = float(f_result.pvalue)

### Tabla A.1 Panel A - Pruebas de balance

Coeficiente del tratamiento con errores estándar clusterizados a nivel de centro de salud. `***` p<0.01, `**` p<0.05, `*` p<0.1. Efectos fijos de provincia incluidos.

In [11]:
def add_stars(pval):
    if pval < 0.01:   return '***'
    elif pval < 0.05: return '**'
    elif pval < 0.1:  return '*'
    return ''

rows = []
for r in results_balance:
    stars = add_stars(r['pval'])
    rows.append({
        'Variable':      r['Variable'],
        'Coeficiente':   f"{r['coef']:.4f}{stars}",
        '(SE)':          f"({r['se']:.4f})",
        'p-valor':       f"{r['pval']:.3f}",
        'Media control': f"{r['mean_ctrl']:.4f}",
        'N':             r['N']
    })

df_display4 = pd.DataFrame(rows).set_index('Variable')

Exporta la tabla de balance en formato LaTeX.

In [12]:
rows_export = []
for r in results_balance:
    stars = add_stars(r['pval'])
    rows_export.append({
        'Variable':      r['Variable'],
        'Coeficiente':   f"{r['coef']:.4f}{stars}",
        '(SE)':          f"({r['se']:.4f})",
        'p-valor':       f"{r['pval']:.3f}",
        'Media control': f"{r['mean_ctrl']:.4f}",
        'N':             str(r['N'])
    })

df_bal_exp = pd.DataFrame(rows_export).set_index('Variable')

latex_str = df_bal_exp.to_latex(
    caption='Balance muestral: características de las mujeres en línea base',
    label='tab:balance',
    escape=False,
    column_format='lrrrrr',
    position='htbp'
)

nota = (
    '\\begin{tablenotes}\\small\n'
    f'  \\item F-estadístico conjunto: {f_stat:.3f} (p-valor: {f_pval:.3f}). '
    '*** p<0.01, ** p<0.05, * p<0.1. '
    'Errores estándar clusterizados a nivel de centro de salud entre paréntesis. '
    'Efectos fijos de provincia incluidos.\n'
    '\\end{tablenotes}\n'
)
latex_str = latex_str.replace('\\end{table}', nota + '\\end{table}')

with open(tables / 'tabla_balance.tex', 'w', encoding='utf-8') as f:
    f.write(latex_str)

---
## Punto 5. Replicación de resultados principales (Tabla 2)

### 5a. Construcción de variables de resultado y merge

Construye el indicador de nacimiento durante el estudio (resultado principal). La variable toma el valor de 1 si la mujer tuvo al menos un nacimiento entre la línea base (abril 2018) y el seguimiento (junio 2021), y 0 en caso contrario. Se excluyen los embarazos que terminaron en aborto o pérdida.

In [13]:
df_e = df_endline.copy()

# Ventana del estudio
start_date = pd.Timestamp('2018-04-01')
end_date   = pd.Timestamp('2021-06-01')

# Indicador: al menos un nacimiento dentro de la ventana del estudio
any_birth = pd.Series(False, index=df_e.index)
for k in range(1, 14):
    dob_c  = f'an_ew_date_dob_{k}'
    miss_c = f'an_ew_preg_is_miscarriage_{k}'
    if dob_c not in df_e.columns:
        continue
    dob      = pd.to_datetime(df_e[dob_c], errors='coerce')
    in_study = (dob >= start_date) & (dob <= end_date)
    not_miss = (df_e[miss_c].fillna(1) == 0) if miss_c in df_e.columns else True
    any_birth = any_birth | (in_study & not_miss)

df_e['any_birth'] = any_birth.astype(float)

### Merge y tasa de atrición

Vincula las bases y calcula la tasa de atrición total y por grupo de tratamiento.

In [14]:
ids_endline  = set(df_e['new_womid_final'])
ids_treat_bl = set(df_b.loc[df_b['arm_free'] == 1, 'new_womid_final'])
ids_ctrl_bl  = set(df_b.loc[df_b['arm_free'] == 0, 'new_womid_final'])

n_bl_total = len(df_b)
n_el_total = len(df_e)
n_treat_bl = len(ids_treat_bl)
n_ctrl_bl  = len(ids_ctrl_bl)
n_treat_el = len(ids_treat_bl & ids_endline)
n_ctrl_el  = len(ids_ctrl_bl  & ids_endline)

attr_total = n_bl_total - n_el_total
attr_treat = n_treat_bl - n_treat_el
attr_ctrl  = n_ctrl_bl  - n_ctrl_el

# Merge: endline como base, variables del baseline
# Meses de uso moderno: 0 para mujeres que nunca usaron anticonceptivos modernos
df_e['months_modern'] = df_e['an_ew_contra_mod_dur_lst3yrs'].fillna(0)

df_merged = (
    df_e[['new_womid_final', 'any_birth',
          'an_ew_contra_modern_lst3y',
          'months_modern',
          'an_ew_coupon_use_s15q7']]
    .merge(
        df_b[['new_womid_final', 'arm_free', 'new_province', 'new_csps']],
        on='new_womid_final', how='inner'
    )
)

### 5b. Modelo econométrico

El efecto ITT del subsidio completo (100 %) relativo al subsidio parcial (10 %) se estima con:

$$Y_i = \beta_0 + \beta_1 D_i + \gamma_{\text{provincia}} + \varepsilon_i$$

**Componentes del modelo:**
- $Y_i$: variable de resultado para la mujer $i$
- $D_i = 1$ si recibió subsidio del 100 %; $D_i = 0$ si recibió subsidio del 10 %
- $\gamma_{\text{provincia}}$: efectos fijos de provincia (diseño estratificado)
- $\varepsilon_i$: término de error
- $\beta_1$: efecto ITT del subsidio completo sobre el resultado

Los errores estándar se clusterizaron a nivel de centro de salud (100 grupos).

### 5c. Estimación del efecto ITT

Estima el modelo para los tres resultados principales del estudio.

In [15]:
province_dummies_m = pd.get_dummies(
    df_merged['new_province'], prefix='prov', drop_first=True
).astype(float)

# Columnas 1-3 de la Tabla 2 del artículo (sin contar columna 4 = vouchers)
outcome_names = {
    'any_birth':                  'Nacimiento durante el estudio',
    'an_ew_contra_modern_lst3y':  'Usó anticonceptivo moderno en 3 años',
    'months_modern':              'Meses de uso moderno (último episodio por método)',
}

# Columna 4 - Por si acaso
outcome_voucher = {
    'an_ew_coupon_use_s15q7': 'Uso de voucher'
}

results_itt = []
for var, label in outcome_names.items():
    model, mask = run_reg_clustered(
        var, df_merged, 'arm_free', province_dummies_m, 'new_csps'
    )
    results_itt.append({
        'Resultado':  label,
        'coef':       model.params['arm_free'],
        'se':         model.bse['arm_free'],
        'pval':       model.pvalues['arm_free'],
        'mean_ctrl':  df_merged.loc[df_merged['arm_free'] == 0, var].mean(),
        'N':          int(mask.sum())
    })

results_voucher = []
for var, label in outcome_voucher.items():
    model_v, mask_v = run_reg_clustered(
        var, df_merged, 'arm_free', province_dummies_m, 'new_csps'
    )
    results_voucher.append({
        'Resultado': label,
        'coef':      model_v.params['arm_free'],
        'se':        model_v.bse['arm_free'],
        'pval':      model_v.pvalues['arm_free'],
        'mean_ctrl': df_merged.loc[df_merged['arm_free']==0, var].mean(),
        'N':         int(mask_v.sum())
    })
    r = results_voucher[-1]
    stars_v = add_stars(r['pval'])
    print(f"Voucher: coef={r['coef']:.4f}{stars_v} ({r['se']:.4f}), ctrl={r['mean_ctrl']:.4f}, N={r['N']}")

Voucher: coef=0.0382*** (0.0117), ctrl=0.1346, N=7472


### Tabla 2 - Efectos ITT del subsidio sobre resultados principales

Errores estándar clusterizados a nivel de centro de salud entre paréntesis. `***` p<0.01, `**` p<0.05, `*` p<0.1. Efectos fijos de provincia incluidos.

In [16]:
rows_itt = []
for r in results_itt:
    stars = add_stars(r['pval'])
    rows_itt.append({
        'Resultado':     r['Resultado'],
        'Coeficiente':   f"{r['coef']:.4f}{stars}",
        '(SE)':          f"({r['se']:.4f})",
        'p-valor':       f"{r['pval']:.3f}",
        'Media control': f"{r['mean_ctrl']:.4f}",
        'N':             r['N']
    })


Exporta la Tabla 2 en formato LaTeX.

In [17]:
rows_itt_exp = []
for r in results_itt:
    stars = add_stars(r['pval'])
    rows_itt_exp.append({
        'Resultado':     r['Resultado'],
        'Coeficiente':   f"{r['coef']:.4f}{stars}",
        '(SE)':          f"({r['se']:.4f})",
        'p-valor':       f"{r['pval']:.3f}",
        'Media control': f"{r['mean_ctrl']:.4f}",
        'N':             str(r['N'])
    })

df_itt_exp = pd.DataFrame(rows_itt_exp).set_index('Resultado')

latex_str_itt = df_itt_exp.to_latex(
    caption='Efectos ITT del subsidio completo sobre resultados principales',
    label='tab:itt',
    escape=False,
    column_format='lrrrrr',
    position='htbp'
)

nota_itt = (
    '\\begin{tablenotes}\\small\n'
    '  \\item *** p<0.01, ** p<0.05, * p<0.1. '
    'Errores estándar clusterizados a nivel de centro de salud entre paréntesis. '
    'Efectos fijos de provincia incluidos.\n'
    '\\end{tablenotes}\n'
)
latex_str_itt = latex_str_itt.replace('\\end{table}', nota_itt + '\\end{table}')

with open(tables / 'tabla_itt.tex', 'w', encoding='utf-8') as f:
    f.write(latex_str_itt)